In [1]:
# !conda install --no-deps -c menpo opencv=2.4.11 --yes

In [2]:
# ! pip uninstall matplotlib --yes

In [3]:
# ! pip install matplotlib -q

In [4]:
import time 
import requests
import cv2
import operator
import numpy as np
import json

# Import compatibility libraries (python 2/3 support)
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

# Python 3
try:
    from urllib.request import urlopen, Request
    from urllib.parse import urlparse, urlencode
    from http.client import HTTPSConnection
# Python 2.7
except ImportError:
    from urlparse import urlparse
    from urllib import urlencode
    from urllib2 import Request, urlopen
    from httplib import HTTPSConnection

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 
# Display images within Jupyter

In [10]:
# This could also be an external file
config_json = '''{
"subscription_key_computer_vision": "7a4c8dfe22a246a3b0ca045c26d8621c"
}'''

CONFIG=json.loads(config_json)
# grab key from json in config
subscription_key = CONFIG['subscription_key_computer_vision']

In [11]:
# Make sure to take your Endpoint from Overview page of API in Azure Portal
_url = 'https://southcentralus.api.cognitive.microsoft.com/customvision/v2.0/Prediction/98f8b1d4-53d2-4bd9-a6f6-caac5b39c0ff/url?iterationId=a96ddf60-bc4e-4d7d-b132-dc058e81bf0d' # change the ending to 'analyze' or 'describe'
_key = subscription_key

_maxNumRetries = 10

In [12]:
def processRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:
        # Use the requests library to make the POST call
        response = requests.request( 'post', 
                                    _url, 
                                    json=json, 
                                    data=data, 
                                    headers=headers, 
                                    params=params )

        if response.status_code == 429: 
            print( "Message: %s" % ( response.json()['error']['message'] ) )
            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break
                
        # We have a successful response, but let's do some tests on response data
        elif response.status_code == 200 or response.status_code == 201:
            if 'content-length' in response.headers and \
                int(response.headers['content-length']) == 0: 
                    
                result = None
                
            elif 'content-type' in response.headers and \
                isinstance(response.headers['content-type'], str):
                    
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None
                    
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json() ) )

        break
        
    return result

In [13]:
def renderResultOnImage( result, img ):
    """Display the obtained results onto the input image"""

    if 'description' in result:
        descrip = result['description']['captions'][0]['text']
        print(descrip)
        cv2.putText(img, descrip, (30,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 3)
    if 'categories' in result:
        # Put a rectangle around the image to reflect accent colors
        R = int(result['color']['accentColor'][:2],16)
        G = int(result['color']['accentColor'][2:4],16)
        B = int(result['color']['accentColor'][4:],16)
        if img is not None:
            cv2.rectangle(img,(0,0), (img.shape[1], img.shape[0]), color=(R,G,B), thickness=25)

        categoryName = sorted(result['categories'], key=lambda x: x['score'])[0]['name']
        print(categoryName)
        cv2.putText(img, categoryName, (30,70), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 3)

In [14]:
urlImage = 'https://outdoorgearlab-mvnab3pwrvp3t0.stackpathdns.com/photos/16/51/286670_31243_M.jpg'

In [15]:
from IPython.display import display, Image


# Computer Vision parameters
params = { 'visualFeatures' : 'Color,Categories'} 

headers = dict()
headers['Prediction-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 'url': urlImage } 
data = None

result = processRequest(json, data, headers, params)

if result is not None:
    # Load the original image, fetched from the URL
    arr = np.asarray( bytearray( requests.get( urlImage ).content ), 
                     dtype=np.uint8 )
    arr = arr[...,::-1]
    
    img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
    
    

    renderResultOnImage(result, img)
    
    if img is not None:
        plt.subplots(figsize=(15, 20))
        plt.imshow(img)
    else:
        img = Image(url=urlImage, embed=False)
        display(img)

In [17]:
print(result)

{'predictions': [{'tagName': 'hardshell_jackets', 'tagId': '42b3d053-5bbc-4de8-8387-be2a0700a437', 'probability': 1.0}, {'tagName': 'insulated_jackets', 'tagId': 'f768db76-1fdd-4441-9984-a81b2c397c98', 'probability': 5.538676e-08}], 'iteration': 'a96ddf60-bc4e-4d7d-b132-dc058e81bf0d', 'id': '9c4b89da-1741-413f-a268-94e4a2c56b8f', 'project': '98f8b1d4-53d2-4bd9-a6f6-caac5b39c0ff', 'created': '2018-10-02T18:22:16.2574208Z'}
